# Distributed AMFabric

In [ ]:
import os
from dotenv import load_dotenv

# setup configuration
#
load_dotenv()
config = {'db_name': os.getenv("DB_NAME"),
              'db_username': os.getenv("DB_USERNAME"),
              'db_password': os.getenv("DB_PASSWORD"),
              'db_system': os.getenv("DB_SYSTEM"),
              'db_config_file_path': os.getenv("DB_CONFIG_PATH"),
              'db_queries_file_path': os.getenv("DB_QUERIES_PATH"),
              'fabric_graph' : 'HSG',
              'scheduler_address': os.getenv("DASK_SCHEDULER")}

### Instantiate Distributed AMFabric object

In [ ]:
from src.distributed_amfabric import DistributedAMFabric

amf = DistributedAMFabric(config=config)


### Create some simple training data use an SDR

In [ ]:
from src.sdr import SDR
sdr_1 = SDR()
sdr_1.set_item(source_node = ('rainbow_trade', '*'),
               edge=('has_rgb', 'r'),
               target_node=('rgb', 'r'),
               probability=1.0,
               numeric=125,
               numeric_min=0,
               numeric_max=255
              )

sdr_1.sdr


### Create an area in the fabric to train

In [ ]:
# the length of the short term memeory - defines number of neurons in a neuro_column
#
short_term_memory = 1

# multiplier to determine the length of lookback (mp_threshold * short_term_memory) when looking for anomalies and motifs
#
mp_threshold = 6

# the strategy for connecting neuro_columns
#
structure = 'star'

# edges with probability below this threshold will be assumed to be zero and deleted
#
prune_threshold = 0.001

# a seed to ensure we can repeat our experiments
#
random_seed = 221166

# create a new amfabric area
#
amf.create_distributed_fabric(fabric_uid='test',
                              short_term_memory=short_term_memory,
                              mp_threshold=mp_threshold,
                              structure=structure,
                              prune_threshold=prune_threshold,
                              random_seed=random_seed,
                              restore=True)

### Search for the neuro_column most similar to the SDRs in short term memory

In [ ]:
search_por = amf.search_for_bmu(fabric_uid='test', sdr=sdr_1, ref_id=1)
search_por

In [ ]:
search_por = search_por.result()
search_por

### Learn the current SDRs in short term memory, using the results from the previous search

In [ ]:
learn_por = amf.learn(fabric_uid='test', search_por=search_por, persist=True)
learn_por

In [ ]:
learn_por = learn_por.result()
learn_por

### Query the fabric - basically a search for the most similar neuro_column to the SDRs in the query

In [ ]:
q_result = amf.query(fabric_uid='test', sdr=sdr_1)
q_result

In [ ]:
q_result = q_result.result()
q_result

### Decode the fabric

In [ ]:
decode_result = amf.decode_fabric(fabric_uid='test', all_details=True,community_sdr=True)
decode_result

In [ ]:
decode_result = decode_result.result()
decode_result